In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from botorch.models.transforms.input import Normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import ExpectedImprovement
from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.acquisition import qLogExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.likelihoods import GaussianLikelihood
import gpytorch
import win32com.client
from botorch.models.transforms import Normalize
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.metrics import mean_absolute_error
from scipy.stats import qmc
import os
import subprocess
from pathlib import Path
import time
import pandas as pd
import plotly.graph_objects as go
from utils import parse_real_disp, CABC_Displacement, CABC_Force, parse_real_force

torch.set_default_dtype(torch.float64)

# Random Parameter Generation

In [2]:
# Bounds
n_bounds = (0.0, 1.0)
tau_star_bounds = (400000,0, 7000000.0)
# B_bounds_log = (np.log10(1.0e+14), np.log10(1.916e+19))  # log10 scaling
tau_w_bounds = (0.001, 1.0)
m_bounds = (0.1, 1.9)

# LHs Sampling
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(10)  # 5 samples

# # Scaling
n_values = sample[:, 0] * (n_bounds[1] - n_bounds[0]) + n_bounds[0]
tau_star_values = sample[:, 1] * (tau_star_bounds[1] - tau_star_bounds[0]) + tau_star_bounds[0]
# B_values = 10 ** (sample[:, 2] * (B_bounds_log[1] - B_bounds_log[0]) + B_bounds_log[0])
tau_w_values = sample[:, 2] * (tau_w_bounds[1] - tau_w_bounds[0]) + tau_w_bounds[0]
m_values = sample[:, 3] * (m_bounds[1] - m_bounds[0]) + m_bounds[0]

# # save2CSV
df = pd.DataFrame({
    'n': n_values,
    'tau_star': tau_star_values,
    # 'B': B_values,
    'tau_w': tau_w_values,
    'm': m_values
})
df.to_csv("combined_LHS_parameters.csv", index=False)

# XML Parsing - Master Curve

In [3]:
# Directories
real_disp_path = "H:\\Real_data\\press_displacement_real.xml"
real_force_path = "H:\\Real_data\\press_force_real.xml"

# Displacement XML

# real_disp_time, real_disp = parse_real_disp(real_disp_path)

# real_tree = ET.parse(real_disp_path)
# real_root = real_tree.getroot()
# real_data = []
# guess_data_all = {}

# for time_block, dept_block in zip(real_root.findall('.//Block//IndpVar'), real_root.findall('.//Block')):

#     time_value = time_block.attrib.get('Value')
#     dept_value = dept_block.find('DeptValues').text.strip()
#     real_data.append((time_value, dept_value))

# real_data_array = np.array([(float(x), float(y)) for x, y in real_data])
# real_time = real_data_array[:, 0]  
# real_disp = real_data_array[:, 1] * 0.0001019716


# Force XML

real_force_time, real_force = parse_real_force(real_force_path)

def calculate_area_between_curves(real_time, real_disp, guess_time, guess_disp):
    common_time = np.linspace(0.0, max(real_time.max(), guess_time.max()), num=200)

    # Resample both curves on this time grid
    real_disp_resampled = np.interp(common_time, real_time, real_disp)
    guess_disp_resampled = np.interp(common_time, guess_time, guess_disp)

    mse = np.mean((real_disp_resampled-guess_disp_resampled)**2)
    
    return mse

real_data_directory = "H:\\Real_data\\press_force_real.xml"
areas = []
area_data = []

real_tree = ET.parse(real_data_directory)
real_root = real_tree.getroot()
real_data = []
guess_data_all = {}

count = 0

for time_block, dept_block in zip(real_root.findall('.//Block//IndpVar'), real_root.findall('.//Block')):
    if count >=  27:
        break  # Stop the loop after collecting 27 values

    time_value = time_block.attrib.get('Value')
    dept_value = dept_block.find('DeptValues').text.strip()
    real_data.append((time_value, dept_value))

    count += 1

real_data_array = np.array([(float(x), float(y)) for x, y in real_data])
real_time = real_data_array[:, 0]  
real_disp = real_data_array[:, 1]   / 1000

In [4]:
# file_info = [
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKBAR150bar03mms388g.txt",  #0
#         "force_start"   : "31.07.2025 10:39:06.150461",
#         "force_end"     : "31.07.2025 10:39:24.070461",
#         "pos_start"     : "31.07.2025 10:39:24.070461",
#         "pos_end"       : "31.07.2025 10:42:34.610461"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKBAR150bar07mms384g.txt",  #1
#         "force_start"   : "31.07.2025 10:18:27.193997",
#         "force_end"     : "31.07.2025 10:18:35.113997",
#         "pos_start"     : "31.07.2025 10:18:35.113997",
#         "pos_end"       : "31.07.2025 10:20:44.573997"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKBAR150bar11mms414g.txt",  #2
#         "force_start"   : "30.07.2025 13:43:44.197899",
#         "force_end"     : "30.07.2025 13:43:49.057899",
#         "pos_start"     : "30.07.2025 13:43:49.057899",
#         "pos_end"       : "30.07.2025 13:46:49.957899"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKCENTER150bar03mms396g.txt", #3
#         "force_start"   : "29.07.2025 15:27:01.321079",
#         "force_end"     : "29.07.2025 15:27:19.181079",
#         "pos_start"     : "29.07.2025 15:27:19.181079",
#         "pos_end"       : "29.07.2025 15:30:46.881079"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKCENTER150bar07mms398g.txt", #4
#         "force_start"   : "30.07.2025 12:19:49.167098",
#         "force_end"     : "30.07.2025 12:19:57.487098",
#         "pos_start"     : "30.07.2025 12:19:57.487098",
#         "pos_end"       : "30.07.2025 12:21:55.287098"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKCENTER150bar11mms393g.txt", #5
#         "force_start"   : "30.07.2025 12:46:38.439119",
#         "force_end"     : "30.07.2025 12:46:43.619119",
#         "pos_start"     : "30.07.2025 12:46:43.619119",
#         "pos_end"       : "30.07.2025 12:50:18.619119"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\60percent4STACKCENTER200bar11mms365g.txt",#6
#         "force_start"   : "31.07.2025 11:00:47.633201",
#         "force_end"     : "31.07.2025 11:00:53.213201",
#         "pos_start"     : "31.07.2025 11:00:53.213201",
#         "pos_end"       : "31.07.2025 11:06:35.493201"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\80percent4STACKBAR150bar03mms508g.txt", #7
#         "force_start"   : "30.07.2025 11:47:26.451826",
#         "force_end"     : "30.07.2025 11:47:42.371826",
#         "pos_start"     : "30.07.2025 11:47:42.371826",
#         "pos_end"       : "30.07.2025 11:50:53.831826"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\80percent4STACKBAR150bar07mms525g.txt", #8
#         "force_start"   : "31.07.2025 14:21:37.055615",
#         "force_end"     : "31.07.2025 14:21:43.535615",
#         "pos_start"     : "31.07.2025 14:21:43.535615",
#         "pos_end"       : "31.07.2025 14:24:04.975615"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\80percent4STACKBAR150bar11mms531g.txt", #9
#         "force_start"   : "31.07.2025 14:49:31.716399",
#         "force_end"     : "31.07.2025 14:49:35.816399",
#         "pos_start"     : "31.07.2025 14:49:35.816399",
#         "pos_end"       : "31.07.2025 15:01:25.776399"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\80percent4STACKCENTER150bar03mms542g.txt", #10
#         "force_start"   : "31.07.2025 09:07:08.361246",
#         "force_end"     : "31.07.2025 09:07:23.161246",
#         "pos_start"     : "31.07.2025 09:07:23.161246",
#         "pos_end"       : "31.07.2025 09:09:48.641246"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\80percent4STACKCENTER150bar07mms527g.txt", #11
#         "force_start"   : "30.07.2025 15:33:47.945013",
#         "force_end"     : "30.07.2025 15:33:54.465013",
#         "pos_start"     : "30.07.2025 15:33:54.465013",
#         "pos_end"       : "30.07.2025 15:36:35.305013"
#     },
#     {
#         "path" : "H:\\DLR Press Data\\80percent4STACKCENTER150bar11mms543g.txt", #12
#         "force_start"   : "31.07.2025 09:56:19.454633",
#         "force_end"     : "31.07.2025 09:56:23.494633",
#         "pos_start"     : "31.07.2025 09:56:23.494633",
#         "pos_end"       : "31.07.2025 09:58:13.974633"  
#     },
#     {
#         "path" : "H:\\DLR Press Data\\SPLITCONF80percent4STACKBAR150bar03mms518g.txt", #13
#         "force_start"   : "31.07.2025 15:16:03.864139",
#         "force_end"     : "31.07.2025 15:16:18.464139",
#         "pos_start"     : "31.07.2025 15:16:18.464139",
#         "pos_end"       : "31.07.2025 15:20:00.584139"  
#     }

# ]

In [5]:
# # Load the CSV data from the first file_info
# index_master = 10  # or whichever index you want
# master_info = file_info[index_master]

# df_master = pd.read_csv(
#     master_info["path"],
#     encoding='utf-16-le',
#     sep=';',
#     skiprows=1,
#     quotechar='"'
# )

# # Clean and preprocess
# df_master = df_master.rename(columns={
#     'time': 'Time',
#     'dbPr\\diIstKraftGes': 'Force'
# })
# df_master = df_master[['Time', 'Force']]
# df_master['Force'] = df_master['Force'].astype(str).str.replace(',', '.', regex=False)
# df_master['Force'] = pd.to_numeric(df_master['Force'], errors='coerce')
# df_master = df_master[df_master['Time'] != 'sec']
# df_master['Time'] = pd.to_datetime(df_master['Time'], format='%d.%m.%Y %H:%M:%S.%f')

# # Time filter and alignment
# force_start = pd.to_datetime(master_info["force_start"], format='%d.%m.%Y %H:%M:%S.%f')
# force_end = pd.to_datetime(master_info["force_end"], format='%d.%m.%Y %H:%M:%S.%f')
# master_df = df_master[(df_master['Time'] >= force_start) & (df_master['Time'] <= force_end)].copy()
# master_df['Time_seconds'] = (master_df['Time'] - master_df['Time'].iloc[0]).dt.total_seconds()

# # Extract time and force as NumPy arrays
# real_time = master_df['Time_seconds'].to_numpy()
# real_force = master_df['Force'].to_numpy() /  9.80665

# # Optional: resample master curve on a common time grid
# common_time = np.linspace(0.0, real_time.max(), num=200)
# real_force_resampled = np.interp(common_time, real_time, real_force)


In [6]:
df = pd.read_csv('combined_LHS_parameters.csv')

n_values = df['n'].tolist()
tau_star_values = df['tau_star'].tolist()
# B_values = df['B'].tolist()
tau_w_values = df['tau_w'].tolist()
m_values = df['m'].tolist()

# Moldflow Simulation

In [ ]:
Synergy = win32com.client.Dispatch("synergy.Synergy")
Synergy.SetUnits("Metric")
moldflow = "C:\\Program Files\\Autodesk\\Moldflow Insight 2026\\bin"                        #Home D:\\Moldflow Insight 2023\\bin       #WS "C:\\Program Files\\Autodesk\\Moldflow Insight 2023\\bin"  #"G:\\Oskay Sözen\\wall_slip\\wall slip.mpi"


Synergy.OpenProject("G:\\Oskay Sözen\\viscosity\\viscosity.mpi")
Project = Synergy.Project
Project.OpenItemByName("80CENTER_study", "Study")



#Simulation loop for wall slip parameters
for i in range(10):

    n = n_values[i]
    tau_star = tau_star_values[i]
    # B = B_values[i]
    tau_w = tau_w_values[i]
    m = m_values[i]


    print(f"Simulation {i+1} with \nn : {n}\ntau_star : {tau_star}\ntau_w : {tau_w}\nm : {m}")

    study_name = f"study_{i+1}"
    Project = Synergy.Project

    # Viscosity Parameters

    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090,1)
    Prop.FieldDescription(53030, f"{n}, {tau_star}, 1.916e+19, 0.004, 0, 0, 0.305")
    PropEd.CommitChanges("Process Conditions")

    # Wall Slip Parameters

    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090, 1)
    DVec = Synergy.CreateDoubleArray
    
    DVec.AddDouble(tau_w)
    DVec.AddDouble(m)
    DVec.AddDouble(1e-08)
    DVec.AddDouble(0)
    DVec.AddDouble(0)
    
    Prop.FieldValues(51182, DVec)
        
    PropEd.CommitChanges("Process Conditions")

    StudyDoc = Synergy.StudyDoc
    StudyDoc.SaveAs(study_name)

    p = subprocess.Popen(
    [os.path.join(moldflow, "runstudy.exe"), f"G:\\Oskay Sözen\\viscosity\\study_{i+1}.sdy"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    )
    (output, err) = p.communicate()

    time.sleep(3)

    # Restart of the Synergy to prevent XML saving crash
    
    subprocess.run(["taskkill", "/f", "/im", "airsyn_synergy.exe"]) 

    time.sleep(3)

    Synergy.OpenProject("G:\\Oskay Sözen\\viscosity\\viscosity.mpi")         # PROJECT NAME!!!!!!!!!!!!!!!!
    Project = Synergy.Project
    Project.OpenItemByName(f"study_{i+1}", "Study")

    # Plot Export

    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    Plot = PlotMgr.FindPlotByName2("Press force:XY Plot", "Press force")
    Pid = Plot.GetDataID
    PlotMgr = None
    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    PlotMgr.SaveResultDataInXML2( Pid, f"H:\\XMLFiles\\study_{i + 1}_force.xml", "Metric")

    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    Plot = PlotMgr.FindPlotByName2("Press displacement:XY Plot", "Press displacement")
    Pid = Plot.GetDataID
    PlotMgr = None
    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    PlotMgr.SaveResultDataInXML2( Pid, f"H:\\XMLFiles\\study_{i + 1}_disp.xml", "Metric")


# Write the output to a log file
    with open(f"H:\\Log_Files\\study_{i+1}.log", "w") as file:
        file.write(output.decode("windows-1252").strip())



Simulation 1 with 
n : 0.9416036385905784
tau_star : 153612.03670543525
tau_w : 0.2548863537027709
m : 0.6004306096626426
Simulation 2 with 
n : 0.8038895180823131
tau_star : 195649.4594493557
tau_w : 0.736742292379642
m : 1.6593989032112682
Simulation 3 with 
n : 0.0917068964595216
tau_star : 323623.8235745369
tau_w : 0.048330410534807
m : 0.4025302969253785
Simulation 4 with 
n : 0.5411172089386926
tau_star : 47553.990305868734
tau_w : 0.3511112110673176
m : 0.7006687681587517
Simulation 5 with 
n : 0.2242629623983043
tau_star : 265011.59908144845
tau_w : 0.186910371693733
m : 1.1394463553283594
Simulation 6 with 
n : 0.4840719321588449
tau_star : 20105.43935102504
tau_w : 0.6775681420688854
m : 1.861729504942771
Simulation 7 with 
n : 0.6232496450207361
tau_star : 295185.9591318524
tau_w : 0.5899878389168749
m : 1.5202707300506155
Simulation 8 with 
n : 0.333950301072446
tau_star : 113977.2162116709
tau_w : 0.4107290512975221
m : 0.9959870196379448
Simulation 9 with 
n : 0.771386775

# XML Parsing - Guess Curves

In [ ]:
guess_disp_data_all = {}
guess_force_data_all = {}
area_data = []
guess_data_all = {}

valid_xml = []
for i in range(10):
        guess_tree = ET.parse(f"H:\\XMLFiles\\study_{i + 1}_force.xml")
        guess_root = guess_tree.getroot()
        guess_data = []

        count = 0
                
        for time_block, dept_block in zip(guess_root.findall('.//Block//IndpVar'), guess_root.findall('.//Block')):
                if count >= 35:
                        break


                time_value = time_block.attrib.get('Value')
                dept_value = dept_block.find('DeptValues').text.strip()
                
                guess_data.append((time_value, dept_value))

                count += 1
        if len(guess_data) < 27:
                print(f"Warning: File {i+1} has incomplete or corrupted data (only {len(guess_data)} entries). Skipping.")
                continue

        guess_data_all[f"guess_data_{i+1}"] = guess_data

        guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
        guess_time = guess_data_array[:, 0]  
        guess_disp = guess_data_array[:, 1] * 9.80665

        globals()[f"guess_time_{i+1}"] = guess_time
        globals()[f"guess_disp_{i+1}"] = guess_disp 


        mean_s_error = calculate_area_between_curves(real_time, real_disp, guess_time, guess_disp)
        area_data.append(mean_s_error)
        valid_xml.append(len(area_data) - 1)

        
print(guess_data_all)

In [ ]:
print(area_data)
print(list(guess_data_all.keys()))
for name, lst in guess_data_all.items():
    array = np.array(lst)
    print(f"{name}: Shape = {array.shape}")

In [ ]:
common_time = np.linspace(0.0, max(real_time.max(), guess_time.max()), num=200)
real_force_resampled = np.interp(common_time, real_time, real_disp)

num_curves = len(guess_data_all)
colormap = plt.get_cmap('viridis')

plt.figure(figsize=(10, 6))

for i, (key, guess_data) in enumerate(guess_data_all.items(), start=1):
    if isinstance(guess_data[0], (list, tuple)) and len(guess_data[0]) == 2:
        guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
        guess_time = guess_data_array[:, 0]
        guess_force = guess_data_array[:, 1] * 9.80665

    guess_force_resampled = np.interp(common_time, guess_time, guess_force)
    color = colormap((i - 1) / max(num_curves - 1, 1))
    plt.plot(common_time, guess_force_resampled, color=color, label=f'Guess_{i}')

plt.plot(common_time, real_force_resampled, 'b-', label='Real Curve', linewidth=2)

plt.xlabel('Time (s)')
plt.ylabel('Force (kN)')
plt.title('Force vs Time (Guess vs Real)')
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
plt.grid(True)
plt.tight_layout()
plt.show()


# GP Model

In [ ]:
file_path = 'H:\\moldflow\\moldflow_automation\\combined_LHS_parameters.csv'
data = pd.read_csv(file_path)
n_values = data['n'].values
tau_star_values = data['tau_star'].values
# B_values = data['B'].values
tau_w_values = data['tau_w'].values
m_values = data['m'].values

train_x = torch.tensor([[n_values[i], tau_star_values[i], tau_w_values[i], m_values[i]] for i in valid_xml ], dtype=torch.float64)
train_y = - torch.tensor([[area_data[i]] for i in valid_xml], dtype=torch.float64)
train_yvar = torch.full_like(train_y, 1e-6)



bounds = torch.tensor([
    [0.1, 500.0, 0.001, 0.1],
    [1.0, 8000.0, 1.0, 1.9]
], dtype= torch.float64
)

outcome_transform = Standardize(m=1)
input_transform = Normalize(d=4)

# GP Model 
model = SingleTaskGP(train_x, train_y,train_Yvar=train_yvar, outcome_transform=outcome_transform, input_transform=input_transform)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)
model.eval()  


acq_func =  LogExpectedImprovement(model, best_f=train_y.max().item())

candidate, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=1,
        num_restarts=1000, #Helps to ensure that we find the global optimum instead of finding the local optimum
        raw_samples=5000,  #More samples increases likelihood of finding good starting point.
)

cand = model(input_transform(candidate))
cand_var = torch.diagonal(cand.covariance_matrix, 0)
cand_mean, cand_std = outcome_transform.untransform(cand.mean, cand_var)


In [ ]:
print(candidate)
print(train_y)
print(train_x)
bounds.shape

# Optimization CHECK CANDIDATE


In [ ]:
guess_data_all_optim = {}
num_iteration = 5
iterations=[]

for i in range(num_iteration):
    outcome_transform = Standardize(m=1)
    input_transform = Normalize(d=5)

    model = SingleTaskGP(train_x, train_y,train_Yvar=train_yvar, outcome_transform=outcome_transform, input_transform=input_transform)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)
    model.eval()  

    acq_func =  LogExpectedImprovement(model, best_f=train_y.max())


    candidate_optim, _ = optimize_acqf(
            acq_function=acq_func,
            bounds=bounds,
            q=1,
            num_restarts=1000, #Helps to ensure that we find the global optimum instead of finding the local optimum
            raw_samples=5000,  #More samples increases likelihood of finding good starting point.
    )

    print(f"Iteration {i+1}: Suggested new parameters (n, tau_star, B, tau_w, m)", candidate_optim)
    iterations.append(candidate_optim)

    study_name = f"optimization_{i+1}"
    Project = Synergy.Project

    # Viscosity Parameters

    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090,1)
    Prop.FieldDescription(53030, f"{candidate_optim[0,0].item()}, {candidate_optim[0,1].item()}, 1e+08, 0.004, 0, 0, 0.305")
    PropEd.CommitChanges("Process Conditions")

    # Wall Slip Parameters

    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090, 1)
    DVec = Synergy.CreateDoubleArray
    
    DVec.AddDouble(candidate_optim[0,2].item())
    DVec.AddDouble(candidate_optim[0,3].item())
    DVec.AddDouble(1e-08)
    DVec.AddDouble(0)
    DVec.AddDouble(0)
    
    Prop.FieldValues(51182, DVec)
        
    PropEd.CommitChanges("Process Conditions")

    StudyDoc = Synergy.StudyDoc
    StudyDoc.SaveAs(study_name)

    p = subprocess.Popen(
    [os.path.join(moldflow, "runstudy.exe"), f"G:\\Oskay Sözen\\viscosity\\optimization_{i+1}.sdy"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    )
    (output, err) = p.communicate()

    time.sleep(3)

    Synergy.OpenProject("G:\\Oskay Sözen\\viscosity\\viscosity.mpi")         # PROJECT NAME!!!!!!!!!!!!!!!!
    Project = Synergy.Project
    Project.OpenItemByName(f"optimization_{i+1}", "Study")

    # Plot Export

    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    Plot = PlotMgr.FindPlotByName2("Press force:XY Plot", "Press force")
    Pid = Plot.GetDataID
    PlotMgr = None
    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    PlotMgr.SaveResultDataInXML2( Pid, f"H:\\XMLFiles\\optimization_{i + 1}.xml", "Metric")



# Write the output to a log file
    with open(f"H:\\Log_Files\\optimization_{i+1}.log", "w") as file:
        file.write(output.decode("windows-1252").strip())

    guess_tree = ET.parse(f"H:\\XMLFiles\\optimization_{i+1}.xml")
    guess_root = guess_tree.getroot()
    guess_data = []
    time.sleep(1) 
    count = 0   
    for time_block, dept_block in zip(guess_root.findall('.//Block//IndpVar'), guess_root.findall('.//Block')):
        if count >= 30:
            break
        
        time_value = time_block.attrib.get('Value')
        dept_value = dept_block.find('DeptValues').text.strip()
        guess_data.append((time_value, dept_value))

        count += 1
    if len(guess_data) < 30:
        print(f"Warning: File {i+1} has incomplete or corrupted data (only {len(guess_data)} entries). Skipping.")
        continue
    guess_data_all_optim[f"guess_data_{i+1}"] = guess_data

    guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
    guess_time = guess_data_array[:, 0]  
    guess_disp = guess_data_array[:, 1]  
        
    globals()[f"guess_time_{5 + num_iteration}"] = guess_time
    globals()[f"guess_disp_{5 + num_iteration}"] = guess_disp
    
    mean_s_error = calculate_area_between_curves(real_time, real_disp, guess_time, guess_disp)
    area_data.append(mean_s_error)
    valid_xml.append(len(area_data) - 1)

    # New Values

    train_x = torch.cat([train_x, candidate_optim], dim=0)
    train_y = torch.cat([train_y, -torch.tensor([[area_data[-1]]])], dim=0)
    train_yvar = torch.full_like(train_y, 1e-6)

    current_best_area = train_y.max().item()

    if abs(current_best_area) < 1e-2:
        print(f"Converged after {i+1} iteration")
        break
    
best_params = train_x[train_y.argmax()]
best_area = train_y.max().item()
print(f"Best parameters (n, tau_star, B, tau_w, m): {best_params}, with MSE : {best_area}")





In [ ]:
num_curves = len(guess_data_all)
colormap = plt.get_cmap('viridis')
for i, (key, guess_data) in enumerate(guess_data_all.items(), start=1):
    if isinstance(guess_data[0], (list, tuple)) and len(guess_data[0]) == 2:
        guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
        guess_time = guess_data_array[:, 0]
        guess_disp = guess_data_array[:, 1]

    guess_disp_resampled = np.interp(common_time, guess_time, guess_disp)

    
    color = colormap((i - 1) / max(num_curves - 1, 1))
    plt.plot(common_time, guess_disp_resampled, color=color, label=f'Guess_{i}')


real_disp_resampled = np.interp(common_time, real_time, real_disp)
plt.plot(common_time, real_disp_resampled, 'b-', label='Real Curve', linewidth=2)

# Finalize plot
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
plt.xlabel('Time (s)')
plt.ylabel('Force')
plt.title('Force vs Time (5 Random Guess Curves)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

num_curves = len(guess_data_all_optim)
colormap = plt.get_cmap('viridis', num_curves)  
plt.plot(common_time, real_disp_resampled, 'b-', label='Real Curve')
for i, (key, guess_data) in enumerate(guess_data_all_optim.items()):
    if isinstance(guess_data[0], (list, tuple)) and len(guess_data[0]) == 2:
        guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
        guess_time = guess_data_array[:, 0]
        guess_disp = guess_data_array[:, 1]

        guess_disp_resampled = np.interp(common_time, guess_time, guess_disp)

        color = colormap(i) 
        plt.plot(common_time, guess_disp_resampled, color=color, label=f'Guess_{i+1}')

# Finalize plot
plt.legend(loc='best')
plt.xlabel('Time (s)')
plt.ylabel('Force')
plt.title('Force vs Time (Optimized Curves)')
plt.grid(True)
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
plt.tight_layout()  
plt.show()